In [0]:
!pip install errant

In [101]:
!pip install tqdm

In [52]:
!git clone https://github.com/samueljamesbell/m2-correct

Cloning into 'm2-correct'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), done.


First task: we could take the word that is the max suggest replacement AND in the same word set
- mask each token one at a time
- determine which one is the error based on the confidence
- make the replacement
- check if we got it right

Second task: 

- same thing but doesn't know, so we replace as many across a certain threshold
- shows how good bert is correcting for an overall replacement for a sentence (more than single edit?)

- same thing but we do it for each word, above a maximum confidence (above 30? or 40?)

What we want overall: Handle data from our original data file, flattened. Takes wrong sentence, provides suggestions, makes the correction, and compares them to the correct sentence




In [0]:
!pip install transformers

In [0]:
import torch
from transformers import pipeline, BertTokenizer, BertModel, BertForMaskedLM
import sys, string

In [0]:
import logging
logging.basicConfig(level=logging.INFO)
import torch
from operator import itemgetter

In [0]:
def fstr(sentence):
    return eval(f"f'{sentence}'")

def mask(sentence, masked_index):
    return sentence[:masked_index] + ['[MASK]'] + sentence[masked_index+1:]

fill_mask = pipeline(
    "fill-mask",
    model="bert-base-uncased",
    tokenizer="bert-base-uncased")

In [37]:
print(fill_mask(f"HuggingFace is creating a [MASK] that the community uses to solve NLP tasks."))

[{'sequence': '[CLS] huggingface is creating a tool that the community uses to solve nlp tasks. [SEP]', 'score': 0.3885316550731659, 'token': 6994}, {'sequence': '[CLS] huggingface is creating a framework that the community uses to solve nlp tasks. [SEP]', 'score': 0.0572054386138916, 'token': 7705}, {'sequence': '[CLS] huggingface is creating a database that the community uses to solve nlp tasks. [SEP]', 'score': 0.048818349838256836, 'token': 7809}, {'sequence': '[CLS] huggingface is creating a program that the community uses to solve nlp tasks. [SEP]', 'score': 0.04593093693256378, 'token': 2565}, {'sequence': '[CLS] huggingface is creating a software that the community uses to solve nlp tasks. [SEP]', 'score': 0.04023849964141846, 'token': 4007}]


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
sent1 = 'HuggingFace is creating an tool that the community uses to solve NLP tasks.'
sent2 = "My town is a medium size city with eighty thousand inhabitants ."
sent3 = "It has a high - density population because its small territory ."
sent4 = "I recommend visiting the artificial lake in the certer of the city which is surrounded by a park ."
sent5 = "The quality of the products and services is quite good , because there is huge competition . However recommend you be careful about some fakes or cheats ."

data = {'sentence': sent5, 'correction': '', 'index': []}
tokenized_sentence = tokenizer.tokenize(data['sentence'])

In [0]:
top_k_pred = 5

In [0]:
def mask_predict_token(tokenized_sentence):
    best_predictions = []
    orignal_sentence = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokenized_sentence))

    for i, word in enumerate(tokenized_sentence):
        if tokenized_sentence[i] in string.punctuation: #avoid calculating punctuation which will likely have the highest result
            continue;

        masked_sentence = tokenizer.convert_tokens_to_string(mask(tokenized_sentence, i)).replace(' ##', '')

        #Note for later: make sure these partial tokens (##) won't cause any problems

        #ranked 1 prediction
        prediction = fill_mask(masked_sentence)[0]

        #removing special tokens
        tokenized_prediction = tokenizer.tokenize(prediction['sequence']) #tokenizing prediction sentence                                      
        no_special_tokens_sentence = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokenized_prediction), skip_special_tokens=True) #actual sentence

        if (orignal_sentence != no_special_tokens_sentence):
            prediction['no_special_token_sentence'] = no_special_tokens_sentence
            best_predictions.append(prediction)

        #print(word, prediction)

    sorted_best_predictions = sorted(best_predictions, key=itemgetter('score'), reverse=True) 
    
    highest_confidence_corrections = []
    for k in range(min(top_k_pred, len(orignal_sentence))):
        highest_confidence_corrections.append(sorted_best_predictions[k])

    top_k_tokens = []
    for ele in highest_confidence_corrections:
        top_k_tokens.append(tokenizer.convert_ids_to_tokens(ele['token']))

    # for ele in sorted_best_predictions:
    #     print(ele)

    #toke sentence with highest confidence correction without special tokens
    return(top_k_tokens)

In [165]:
mask_predict_token(tokenized_sentence)

[',', 'and', 'fake', 'cheat', 'i']

-Now using output3.txt from the AP-GEC github.

-Evaluating sentences from the edits using the m2-correct.

Note: Does not identify spelling errors - so at the moment I am skipping them

In [0]:
preprocessed_file = open('output3.txt')
preprocessed_file_str = preprocessed_file.read()


preprocessed_file_split = preprocessed_file_str.split('\n\n')
sentence_pairs = [ele.split('\n') for ele in preprocessed_file_split]

#remove the 'S' part of the sentences, and extract only the word the edits. Also, only keep replacement edits.
new_sentence_pairs = []
for ele in sentence_pairs:
    try:
        temp0 = ele[0][2:]
        temp1 = ele[1].split("|||")
    except IndexError:
        continue
    
    if (temp1[1][0].lower() != 'r'):
        continue

    #skipping spelling errors as the model fails to identify them
    if (temp1[1][2:].lower() == 'spell'):
        continue

    temp1 = temp1[2].lower()
    new_sentence_pairs.append([temp0, temp1])

In [167]:
#now applying it on the new sentence pairs
from tqdm import tqdm
import string

table = str.maketrans('', '', string.punctuation)

count = 0
correct = 0

N = 15
i = 0

#pbar = tqdm(total=len(new_sentence_pairs))
for pair in new_sentence_pairs:
    sentence = pair[0]

    correct_token = pair[1]
    #removing punctuation because BERT predictions don't have punctuation
    correct_token = [w.translate(table) for w in correct_token]
    correct_token = "".join(correct_token)
    correct_token = correct_token.strip()

    data = {'sentence': sentence, 'correction': '', 'index': []}
    tokenized_sentence = tokenizer.tokenize(data['sentence'])
    predicted_tokens = mask_predict_token(tokenized_sentence)

    print(correct_token)
    print(predicted_tokens)

    if correct_token in predicted_tokens:
        correct += 1
        print("yes")
    else:
        print("now")

    print()

    count += 1
    #pbar.update(1)

    i += 1
    if (i == N): 
        break
#pbar.close()

print(correct)
print(count)

sized
['the', 'sized', 'town', 'people', 'several']
yes

although
['being', 'that', 'most', 'being', 'area']
now

of
['a', 'of', 'are', '##ls', 'popular']
yes

is
['is', 'part', 'kinds', 'in', 'main']
yes

kinds
['part', 'in', 'kinds', 'main', 'on']
yes

businesses
['part', 'in', 'main', 'on', 'restaurants']
now

grocers
['part', 'in', 'main', 'on', 'restaurants']
now

is
['of', 'is', 'and', 'fake', 'is']
yes

is
['of', 'is', 'and', 'fake', 'any']
yes

however
['of', 'and', 'fake', 'appreciate', 'any']
now

recommend
['and', 'fake', 'of', 'no', 'cheat']
now

be
['of', 'and', 'fake', 'no', 'i']
now

careful
['fake', 'and', 'i', 'no', 'cheat']
now

of
[',', 'and', 'fake', 'cheat', 'i']
now

others
['want', 'people', 'some', 'wanna', 'teacher']
now

6
15


In [0]:
#Now for part 2
def mask_predict_sent(tokenized_sentence):
    best_predictions = []
    orignal_sentence = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokenized_sentence))

    min_confidence = 0.80

    for i, word in enumerate(tokenized_sentence):
        if tokenized_sentence[i] in string.punctuation: #avoid calculating punctuation which will likely have the highest result
            continue;

        masked_sentence = tokenizer.convert_tokens_to_string(mask(tokenized_sentence, i)).replace(' ##', '')

        #Note for later: make sure these partial tokens (##) won't cause any problems

        #ranked 1 prediction
        prediction = fill_mask(masked_sentence)[0]

        #removing special tokens
        tokenized_prediction = tokenizer.tokenize(prediction['sequence']) #tokenizing prediction sentence                                      
        no_special_tokens_sentence = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokenized_prediction), skip_special_tokens=True) #actual sentence

        if (orignal_sentence != no_special_tokens_sentence):
            if prediction['score'] > min_confidence:
                prediction['no_special_token_sentence'] = no_special_tokens_sentence
                prediction['index'] = i
                best_predictions.append(prediction)

                print(word + "/" + tokenizer.convert_ids_to_tokens(prediction['token']))

    for prediction in best_predictions:
        tokenized_sentence[prediction['index']] = tokenizer.convert_ids_to_tokens(prediction['token'])

    sentence = ' '.join(tokenized_sentence).replace(' ##', '')

    # for ele in sorted_best_predictions:
    #     print(ele)

    #toke sentence with highest confidence correction without special tokens
    return(sentence)

In [196]:
#works badly with punctuation and spacing.
#maybe an index issue? Like it's replacing the predicted token at the wrong index?
#hmmm how should we fix it?

sentence = "I have my own plan too but I do n't same to them , I want to become a Journalist ."
data = {'sentence': sentence, 'correction': '', 'index': []}
tokenized_sentence = tokenizer.tokenize(data['sentence'])

new_sent = mask_predict_sent(tokenized_sentence)
print(new_sent)

too/,
do/'
t/the
become/be
i have my own plan , but i ' n ' the same to them , i want to be a journalist .


In [168]:
#generate the annotation for the best edit - depracated
import errant
import spacy

nlp = spacy.load('en')
annotator = errant.load('en', nlp)

orig = annotator.parse(orignal_sentence)
cor = annotator.parse(highest_confidence_correction['no_special_token_sentence'])
edits = annotator.annotate(orig, cor)
for e in edits:
    #print(e.o_start, str(e.o_end) + "|||" + str(e.type) + "|||" + e.c_str) #as according to format of data
    print(e.to_m2())

A 3 4|||R:DET|||a|||REQUIRED|||-NONE-|||0
